In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import spei as si 
import geopandas as gpd
from shapely.geometry import Point

### Import stations data

In [2]:
# Import data and metadata

# Load Metadata 
metadata_path = "//gmvstorage.gmv.es/storage/anin/Groundwater/processed_data/stations_metadadata22_active.xlsx"
meta_df = pd.read_excel(metadata_path)
meta_df.set_index('Station',inplace=True)

# Select the columns that are necessary for this code
meta_df = meta_df.iloc[:, 0:2]

# Load data
df_path = "//gmvstorage.gmv.es/storage/anin/Groundwater/processed_data/stations_data22_active.xlsx"
df = pd.read_excel(df_path)
df = df.iloc[:,1:]
df.set_index('date',inplace=True)

### Import boundaries for spatial distribution

In [3]:
# Read the shapefile
shapefile_path = '//gmvstorage.gmv.es/storage/anin/South_African_secondaryCatchments/Secondary Catchment.shp'
areas = gpd.read_file(shapefile_path)

### Compute the SGI for all the stations

In [4]:
# Import the modified sgi function which includes scaling 
from sgi_tools import compute_sgi

In [5]:
# Define groundwater index scale factor
scale = 3

# Create a new DataFrame to store the SGI values
df_sgi = pd.DataFrame()

# Apply the SGI function to each column of the DataFrame

for column in df.columns:
    df_sgi[column] = compute_sgi(df[column],scale)

# Reset the index & Transpose the DataFrame
dates = df_sgi.index.to_pydatetime()
df_sgi = df_sgi.reset_index(drop = True)
df_sgi = df_sgi.T

C:\Users\ecig\AppData\Local\Temp\ipykernel_22292\3776116787.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_sgi[column] = compute_sgi(df[column],scale)
C:\Users\ecig\AppData\Local\Temp\ipykernel_22292\3776116787.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_sgi[column] = compute_sgi(df[column],scale)
C:\Users\ecig\AppData\Local\Temp\ipykernel_22292\3776116787.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Con

#### Perform the average of the SGI values at each timestep for each polygon in the shapefile

In [6]:
# Join with coordinate information in metadata dataframe
df_latlon = df_sgi.join(meta_df)

# Convert the stations dataframe into a GeoDataFrame by specifying the geometry column with the coordinates:

geometry = [Point(xy) for xy in zip(df_latlon['Longitude'], df_latlon['Latitude'])]
stations_gdf = gpd.GeoDataFrame(df_latlon, geometry=geometry)

# Perform a spatial join between the polygons and the stations to determine which stations fall within each polygon:
stations_by_polygon = gpd.sjoin(areas, stations_gdf, how='inner', op='contains')

# Select the columns based on their numeric names
time_step_columns = [col for col in stations_by_polygon.columns if str(col).isdigit()]

# Group the stations by polygon and time step, and calculate the average for each group:
averages = stations_by_polygon.groupby(['SECONDARY'])[time_step_columns].mean()


c:\Users\ecig\Anaconda3\envs\geoTools\lib\site-packages\IPython\core\interactiveshell.py:3373: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
C:\Users\ecig\AppData\Local\Temp\ipykernel_22292\3605716211.py:10: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None

  stations_by_polygon = gpd.sjoin(areas, stations_gdf, how='inner', op='contains')


In [8]:
averages.T

SECONDARY,A2,A3,A4,A5,A6,A7,A8,A9,B2,B3,...,V7,W1,W2,W3,W4,W5,W7,X1,X2,X3
0,0.600530,0.154300,-0.144058,-9.745845e-02,0.267722,-0.099088,0.320302,0.177324,0.226275,-0.379226,...,-0.219723,0.962875,-1.233495,-0.050126,0.412882,-0.575109,0.341075,0.775938,0.505481,0.489394
1,0.609172,0.171247,-0.015986,-1.348482e-02,0.281047,-0.032706,0.285317,0.147239,0.289735,-0.285751,...,-0.333005,0.962875,-1.512390,-0.050126,0.449841,-0.575109,0.366819,0.905461,0.512561,0.473211
2,0.594764,0.042265,0.015797,-6.938894e-17,-0.115028,-0.256114,0.157328,0.055603,0.254587,-0.322592,...,-0.333005,0.641832,-1.512390,-0.050126,-0.378097,-0.575109,0.092176,0.905461,0.513931,0.510165
3,0.432570,-0.159906,0.023297,1.365006e-02,-0.049941,-0.327695,0.201140,0.136805,0.397650,-0.236368,...,-0.333005,0.641832,-1.233495,-0.050126,-0.335674,-0.575109,0.101327,0.817030,0.510233,0.644428
4,0.371238,0.011045,-0.000292,-4.128067e-02,-0.079652,-0.251947,0.178760,0.148218,0.306734,-0.198607,...,-0.333005,0.667186,-1.233495,-0.035409,-0.366765,-0.575109,0.119024,0.792758,0.487734,0.545244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269,1.367745,-1.165616,-0.257103,4.851945e-01,-0.670931,-1.260338,-0.306319,-0.232227,0.259256,-1.357798,...,-0.575109,0.546581,-1.512390,-1.026877,-1.410109,-0.911003,-1.340130,0.050329,-0.094630,-1.084498
270,1.385168,-1.148864,-0.307276,5.015987e-01,-0.615354,-1.229820,-0.337016,-0.222257,0.258701,-1.301165,...,-0.709103,0.751160,-1.233495,-1.036833,-1.461694,-0.935274,-1.059737,-0.186525,-0.248432,-1.300500
271,1.449085,-0.932336,0.868659,1.407158e+00,0.050530,-0.010797,1.296276,0.170278,0.222397,-1.320561,...,-0.575109,0.780074,-1.233495,-0.697169,-0.648549,-0.905797,1.294080,-0.251106,-0.202839,-1.300500
272,1.777090,2.019086,2.019086,2.019086e+00,2.019086,2.019086,2.019086,2.019086,0.151005,2.019086,...,-0.333005,0.819192,-1.027154,1.346478,2.019086,-0.881526,2.019086,-0.178536,-0.099931,-1.286453


#### Plot the resulting time series by area

In [ ]:

# Iterate through the rows of the averages DataFrame
for index, row in averages.iterrows():
    # Extract the polygon ID from the index
    polygon_id = index

    # Create a pandas time series for the plot function to work
    time_series = pd.Series(row.values, index=dates)
    
    # Create a plot for the current row
    plt.figure()

    # Plot the average values for each time step
    si.plot.si(time_series)

    # Set plot title and labels
    plt.title(f"Catchment: {polygon_id}")
    plt.xlabel("Time ")
    plt.ylabel("SGI")

    # Save the plots
    plt.savefig(f'//gmvstorage.gmv.es/storage/anin/Groundwater/SGI_outputs/TS_plots_scaled/{polygon_id}.png')
